In [ ]:
import tools.globals as globals

a = []

if len(a) == 0:
    print("good")

In [ ]:
#選擇權下單範例

# 步驟
# 取得帳戶資訊
# 確立合約
# 確立order
# 送出order
 
# globals.position = globals.api.list_positions(globals.api.futopt_account)
# Offset.CLOSE: A closing offset (CO) order is a limit order that is executed at market close but can be placed anytime during the trading day. 

import shioaji as sj
from shioaji.constant import Action, OptionRight, StockPriceType, FuturesOrderType, FuturesOCType #選擇權下單，多匯入一個OptionRight常數

api = sj.Shioaji(simulation=True)
api.login(
    person_id='R124743829', 
    passwd='366375Dauhui'
)

#contract = globals.api.Contracts.Options[option_code] #選擇權Contract
contract = api.Contracts.Options['TXO15000L2'] #選擇權Contract
order = api.Order(
    action=Action.Buy,
    price=170, #價格
    quantity=2, #口數
    price_type=StockPriceType.LMT, #MKT: 市價 LMT: 限價
    order_type=FuturesOrderType.ROD,
    octype=FuturesOCType.Cover, #倉別，收盤時平倉
    OptionRight=OptionRight.Call, #選擇權類型
    account=api.futopt_account #下單帳戶指定期貨帳戶
)

trade = api.place_order(contract, order)
print(trade)

In [ ]:
option_code = globals.at_the_money_code +  snap.get_option_code(globals.simulation_optionright)
print(option_code)
globals.contract = positions.fill_contract(option_code)

# Subscribe to the close price of the contract
# contract will be ordered in third BidAsk
# Subscribe to the close price of the contract
globals.api.quote.subscribe(
    globals.contract,
    quote_type = sj.constant.QuoteType.BidAsk, # or 'bidask'
    version = sj.constant.QuoteVersion.v1, # or 'v1'
)

@globals.api.on_bidask_fop_v1()
def quote_callback(exchange:sj.Exchange, bidask:BidAskFOPv1):
    #always avoid making calulations inside the callback function.
    """
    Quoting subscribe function. It is called every tick(theoretically)

    :global param: contract (str)
    :global param: third_best_buy_price (int)
    :global param: third_best_sell_price (int)
    :return: None
    """
    print("ask_price: ", bidask['ask_price'], "bid_price: ", bidask['bid_price'])

    # 取第三檔掛單價格，是為了避免夜盤時價差太大可能造成的風險
    globals.third_best_buy_price = bidask['ask_price'][2] # 第三檔之最佳買價
    globals.third_best_sell_price = bidask['bid_price'][2] # 第三檔之最佳買賣價